In [24]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binom, nbinom, poisson, hypergeom, multinomial
import ipywidgets as widgets
from IPython.display import display, HTML

# Set the style
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['text.color'] = 'black'
plt.rcParams['axes.labelcolor'] = 'black'
plt.rcParams['xtick.color'] = 'black'
plt.rcParams['ytick.color'] = 'black'
plt.rcParams['font.family'] = 'monospace' 
plt.rcParams['font.size'] = 8  # font size
plt.rcParams['font.weight'] = 'normal'  # 'bold', 'light', 'normal'

def format_probability(prob):
    if prob >= 0.001:
        return "{:.4f}".format(prob)
    else:
        return "{:.1e}".format(prob)
    
# Functions to plot each distribution
def plot_pmf_2d(k_values, probs, mean, title, details, x_value=None):
    plt.figure(figsize=(10, 4), dpi=400)
    plt.bar(k_values, probs, alpha=0.6, label='PMF Bars', edgecolor='black', linewidth=0.5, align='center', width=1)
    plt.axvline(mean, color='grey', linestyle='--', label=f'Mean(μ) = {mean:.2f}', linewidth=0.8)
    
    # Creating legend
    handles, labels = plt.gca().get_legend_handles_labels()
    
    if x_value is not None:
        p_less_than_x = np.sum(probs[k_values < x_value])
        p_greater_than_x = np.sum(probs[k_values > x_value])
        plt.scatter(x_value, probs[k_values == x_value], color='red', s=10, zorder=5, label=f'P(X={x_value}) = {format_probability(probs[k_values == x_value][0])}')
        
        # Creating dummy lines for the labels without icons
        line = plt.Line2D([0], [0], linestyle='none')
        handles += [plt.Line2D([0], [0], marker='o', color='red', linestyle='None', markersize=5), line, line]
        labels += [f'P(X={x_value}) = {format_probability(probs[k_values == x_value][0])}',f'P(X<{x_value}) = {format_probability(p_less_than_x)}',
                       f'P(X>{x_value}) = {format_probability(p_greater_than_x)}']
    
    # Reordering handles and labels
    order = [1, 0] + list(range(2, len(handles))) # PMF bars (0) to appear above Mean (1)
    handles = [handles[i] for i in order]
    labels = [labels[i] for i in order]
    
    plt.xlim(0, max(k_values))
    plt.xlabel('X=x')
    plt.ylabel('P(X=x)')
    plt.legend(handles, labels)
    plt.show()
    
    for detail in details:
        print("    " + detail)

def plot_binomial(n, p, x):
    k_values = np.arange(0, n+1)
    probs = binom.pmf(k_values, n, p)
    mean = n * p
    variance = n * p * (1 - p)
    title = 'Binomial PMF'
    details = [
        f"Equation: P(X=x) = C(n, x) * p^x * (1-p)^(n-x)",
        f"Mean (μ) = n * p = {n} * {p:.2f} = {mean:.2f}",
        f"Variance (σ^2) = n * p * (1-p) = {n} * {p:.2f} * {1-p:.2f} = {variance:.2f}",
        f"P(X={x}) = C({n}, {x}) * {p:.2f}^{x} * {(1-p):.2f}^{n-x} = {format_probability(probs[k_values == x][0])}"
    ]
    plot_pmf_2d(k_values, probs, mean, title, details, x)

def plot_nbinom(r, p, x):
    mean = r / p
    variance = r * (1-p) / p**2
    k_values = np.arange(r, mean + max(x, 3*np.sqrt(variance)))
    probs = nbinom.pmf(k_values-r, r, p)
    title = 'Negative Binomial PMF'
    details = [
        f"Equation: P(X=x) = C(x-1, r-1) * p^r * (1-p)^(x-r)",
        f"Mean (μ) = r * (1-p) / p = {r} * {1-p:.2f} / {p:.2f} = {mean:.2f}",
        f"Variance (σ^2) = r * (1-p) / p^2 = {r} * {1-p:.2f} / {p:.2f}^2 = {variance:.2f}",
        f"P(X={x}) = C({x-1}, {r-1}) * {p:.2f}^{r} * {1-p:.2f}^{x-r} = {format_probability(probs[k_values == x][0])}"
    ]
    plot_pmf_2d(k_values, probs, mean, title, details, x)

def plot_hypergeom(N1, N2, n, x):
    N = N1 + N2
    k_values = np.arange(0, min(n+1, N1+1))
    probs = hypergeom.pmf(k_values, N, N1, n)
    mean = n * (N1 / N)
    variance = n * (N1 / N) * ((N - N1) / N) * ((N - n) / (N - 1))
    title = 'Hypergeometric PMF'
    details = [
        f"Equation: P(X=x) = C(N1, x) * C(N2, n-x) / C(N, n)",
        f"Mean (μ) = n * (N1/N) = {n} * {N1/N} = {mean:.2f}",
        f"Variance (σ^2) = n * (N1/N) * ((N-N1)/N) * ((N-n)/(N-1)) = {n} * {N1/N:.4f} * {N-N1/N:.4f} * {N-n/N-1:.4f} = {variance:.2f}",
        f"P(X={x}) = C({N1}, {x}) * C({N2}, {n-x}) / C({N}, {n}) = {format_probability(probs[k_values == x][0])}"
    ]
    plot_pmf_2d(k_values, probs, mean, title, details, x)
    
def plot_poisson(lambd, x):
    k_values = np.arange(0, int(3*lambd))
    probs = poisson.pmf(k_values, lambd)
    mean = lambd
    variance = lambd
    title = 'Poisson PMF'
    details = [
        f"Equation: P(X=k) = e^(-λ) * λ^x / x!",
        f"Mean (μ) = λ = {mean:.1f}",
        f"Variance (σ^2) = λ = {variance:.1f}",
        f"P(X={x}) = e^(-{lambd:.1f}) * {lambd:.1f}^{x} / {x}! = {format_probability(probs[k_values == x][0])}"
    ]
    plot_pmf_2d(k_values, probs, mean, title, details, x)


# Widgets
distribution_dropdown = widgets.Dropdown(
    options=["Select a distribution", "Binomial", "Negative Binomial", "Hypergeometric", "Poisson"],
    value="Select a distribution",
    description='Distribution:'
)

output_container = widgets.VBox([])  # Container to hold sliders and plots
slider_layout = widgets.Layout(width='100%')

def display_distribution_widgets(change):
    if change['new'] == "Binomial":
        n_slider_binom = widgets.IntSlider(value=10, min=1, max=100, step=1, description='n (# of trials):', continuous_update=False, style={'description_width': '200px'}, layout=slider_layout)
        p_slider_binom = widgets.FloatSlider(value=0.5, min=0.05, max=1, step=0.01, description='p (prob. of success):', continuous_update=False, style={'description_width': '200px'}, layout=slider_layout)
        x_slider_binom = widgets.IntSlider(value=5, min=0, max=n_slider_binom.value, step=1, description='x (# of success):', continuous_update=False, style={'description_width': '200px'}, layout=slider_layout)

        def update_x_range_binom(*args):
            x_slider_binom.max = n_slider_binom.value
        
        n_slider_binom.observe(update_x_range_binom, 'value')

        binom_box = widgets.HBox([n_slider_binom, p_slider_binom, x_slider_binom])
        binom_interactive = widgets.interactive(plot_binomial, n=n_slider_binom, p=p_slider_binom, x=x_slider_binom)
        output_container.children = [binom_box, binom_interactive.children[-1]]

    elif change['new'] == "Negative Binomial":
        r_slider_nbinom = widgets.IntSlider(value=5, min=1, max=20, step=1, description='r (# success):', continuous_update=False, style={'description_width': '200px'}, layout=slider_layout)
        p_slider_nbinom = widgets.FloatSlider(value=0.5, min=0.1, max=1, step=0.01, description='p (prob. of success):', continuous_update=False, style={'description_width': '200px'}, layout=slider_layout)
        x_slider_nbinom = widgets.IntSlider(value=6, min=r_slider_nbinom.value, max=25, step=1, description='x (# of trials until r success):', continuous_update=False, style={'description_width': '200px'}, layout=slider_layout)

        def update_x_range_nbinom(*args):
            r = r_slider_nbinom.value
            p = p_slider_nbinom.value
            x = x_slider_nbinom.value
            mean = r / p
            variance = r * (1-p) / p**2
            computed_max = max(int(mean + 4*np.sqrt(variance)), 2*r)
            if r > x_slider_nbinom.max:
                x_slider_nbinom.max = max(r, computed_max)
                x_slider_nbinom.min = r
            else:
                x_slider_nbinom.min = r
                x_slider_nbinom.max = max(r, computed_max)

        r_slider_nbinom.observe(update_x_range_nbinom, 'value')
        p_slider_nbinom.observe(update_x_range_nbinom, 'value')

        nbinom_box = widgets.HBox([r_slider_nbinom, p_slider_nbinom, x_slider_nbinom])
        nbinom_interactive = widgets.interactive(plot_nbinom, r=r_slider_nbinom, p=p_slider_nbinom, x=x_slider_nbinom)
        output_container.children = [nbinom_box, nbinom_interactive.children[-1]]

    elif change['new'] == "Poisson":
        lambd_slider = widgets.FloatSlider(value=5, min=1, max=30, step=0.1, description='λ (average rate):', continuous_update=False, style={'description_width': '200px'}, layout=slider_layout)
        x_slider_poisson = widgets.IntSlider(value=5, min=0, max=int(3*lambd_slider.value - 1), step=1, description='x (# of occurrences):', continuous_update=False, style={'description_width': '200px'}, layout=slider_layout)
        
        def update_x_range_poisson(*args):
            x_slider_poisson.max = int(3*lambd_slider.value - 1)
            
        lambd_slider.observe(update_x_range_poisson, 'value')

        poisson_box = widgets.HBox([lambd_slider, x_slider_poisson])
        poisson_interactive = widgets.interactive(plot_poisson, lambd=lambd_slider, x=x_slider_poisson)
        output_container.children = [poisson_box, poisson_interactive.children[-1]]
    
    elif change['new'] == "Hypergeometric":
        N1_slider = widgets.IntSlider(value=20, min=1, max=100, step=1, description='N1 (# of item in type 1):', continuous_update=False, style={'description_width': '200px'}, layout=slider_layout)
        N2_slider = widgets.IntSlider(value=20, min=1, max=100, step=1, description='N2 (# of item in type 2):', continuous_update=False, style={'description_width': '200px'}, layout=slider_layout)
        n_slider_hypergeom = widgets.IntSlider(value=10, min=1, max=N1_slider.value + N2_slider.value, step=1, description='n (# of picks):', continuous_update=False, style={'description_width': '200px'}, layout=slider_layout)
        x_slider_hypergeom = widgets.IntSlider(value=5, min=0, max=n_slider_hypergeom.value, step=1, description='x (# of picked item from type 1):', continuous_update=False, style={'description_width': '200px'}, layout=slider_layout)

        def update_hypergeom_range(*args):
            n_slider_hypergeom.max = N1_slider.value + N2_slider.value
            x_slider_hypergeom.max = min(n_slider_hypergeom.value, N1_slider.value)
        
        N1_slider.observe(update_hypergeom_range, 'value')
        N2_slider.observe(update_hypergeom_range, 'value')
        n_slider_hypergeom.observe(update_hypergeom_range, 'value')

        hypergeom_box1 = widgets.HBox([n_slider_hypergeom, x_slider_hypergeom])
        hypergeom_box2 = widgets.HBox([N1_slider, N2_slider])
        
        hypergeom_interactive = widgets.interactive(plot_hypergeom, N1=N1_slider, N2=N2_slider, n=n_slider_hypergeom, x=x_slider_hypergeom)
        output_container.children = [hypergeom_box1, hypergeom_box2, hypergeom_interactive.children[-1]]

# Observer
distribution_dropdown.observe(display_distribution_widgets, names='value')

# Display initial dropdown and the output container
display(distribution_dropdown, output_container)

Dropdown(description='Distribution:', options=('Select a distribution', 'Binomial', 'Negative Binomial', 'Hype…

VBox()